# Sciunit Cross-OS Reproducibility Demo on Chameleon

1. Create an **Ubuntu VM** to author a Sciunit experiment
2. Create a **Fedora VM** to reproduce the experiment
3. Use SSH to interact with both machines
4. Demonstrate cross-OS reproducibility using Sciunit

This notebook focuses only on **infrastructure setup**.
Sciunit commands are executed directly on the VMs


In [ ]:
import chi, os, time, datetime
from chi import lease
from chi import server
from chi import context
from chi import network

context.version = "1.0"
context.choose_project()
context.choose_site(default="KVM@TACC")

username = os.getenv("USER")


In [ ]:
exp_name = "sciunit_cross_os_demo"

ubuntu_server_name = f"{exp_name}-ubuntu-{username}"
fedora_server_name = f"{exp_name}-fedora-{username}"
lease_name = f"{exp_name}-lease-{username}"

print("Experiment name:", exp_name)


In [ ]:
l = lease.Lease(
    lease_name,
    duration=datetime.timedelta(hours=6)
)

l.add_flavor_reservation(
    id=server.get_flavor_id("m1.small"),
    amount=2
)

l.submit(idempotent=True)
l.show()


In [ ]:
ubuntu_image = "CC-Ubuntu24.04"

ubuntu_vm = server.Server(
    name=ubuntu_server_name,
    image_name=ubuntu_image,
    flavor_name=l.get_reserved_flavors()[0].name
)

ubuntu_vm.submit(idempotent=True)


In [ ]:
fedora_image = "CC-Fedora39"

fedora_vm = server.Server(
    name=fedora_server_name,
    image_name=fedora_image,
    flavor_name=l.get_reserved_flavors()[0].name
)

fedora_vm.submit(idempotent=True)


In [ ]:
ubuntu_vm.associate_floating_ip()
fedora_vm.associate_floating_ip()

ubuntu_ip = ubuntu_vm.get_floating_ip()
fedora_ip = fedora_vm.get_floating_ip()

print("Ubuntu VM IP:", ubuntu_ip)
print("Fedora VM IP:", fedora_ip)


In [ ]:
sg_list = network.list_security_groups(name_filter="allow-ssh")

if sg_list:
    sg = sg_list[0]
else:
    sg = network.SecurityGroup({
        "name": "allow-ssh",
        "description": "Enable SSH access"
    })
    sg.add_rule("ingress", "tcp", 22)
    sg.submit()

ubuntu_vm.add_security_group(sg.id)
fedora_vm.add_security_group(sg.id)


In [ ]:
ubuntu_vm.check_connectivity()
fedora_vm.check_connectivity()

print("Both Ubuntu and Fedora VMs are ready.")


In [ ]:
print("SSH into Ubuntu (Sciunit author):")
print(f"ssh cc@{ubuntu_ip}")

print("\nSSH into Fedora (Sciunit reproducer):")
print(f"ssh cc@{fedora_ip}")


## Cleanup 

In [ ]:
ubuntu_vm.delete()
fedora_vm.delete()
l.delete()

print("Cleanup commands prepared (commented out).")


_______________________________________________________________________________